# Price prediction of short-term flat rentals in London

* [Data preparation](#Data-preparation)
* [Data pipeline](#Data-pipeline)
* [Predictive modeling](#Predictive-modeling)
* [Cross validation](#Cross-validation)
* [Grid search](#Grid-search)

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

In [2]:
pd.options.display.max_rows = None

In [3]:
home_dir = Path.home()
inside_airbnb_data_dir = home_dir / 'Programming/data/inside-airbnb/london'
inside_airbnb_work_dir = home_dir / 'Programming/Python/machine-learning-exercises/short-term-rents-in-london'

In [4]:
plots_dir = inside_airbnb_work_dir / 'plots'
plots_dir.mkdir(parents=True, exist_ok=True)
hist_dir = plots_dir / 'histograms'
hist_dir.mkdir(parents=True, exist_ok=True)

## Data preparation

In [5]:
inside_airbnb_data_file = inside_airbnb_data_dir / 'selected_short_term_rentals_for_modeling.csv'
inside_airbnb_df = pd.read_csv(inside_airbnb_data_file, keep_default_na=False, thousands=',')

In [6]:
inside_airbnb_df.drop(['room_type', 'nearest_station'], axis=1, inplace=True)
inside_airbnb_df['borough'] = inside_airbnb_df['borough'].replace({r'\s': r'_'}, regex=True)

In [7]:
inside_airbnb_df = inside_airbnb_df.loc[inside_airbnb_df['borough'] != 'Sutton']

In [8]:
inside_airbnb_df['log_price'] = np.log1p(inside_airbnb_df['price'])
inside_airbnb_df = inside_airbnb_df.drop('price', axis=1)

In [9]:
inside_airbnb_df = inside_airbnb_df.drop(['latitude', 'longitude'], axis=1)

In [10]:
inside_airbnb_df.head()

,borough,property_type,bathrooms,bedrooms,minimum_nights,crime_rate,distance_to_station,amenity_1,amenity_2,amenity_3,log_price
0,Lambeth,Entire rental unit,1.0,1.0,30,137.98,0.520193,None,None,None,5.017280
1,Kensington_and_Chelsea,Entire rental unit,2.0,2.0,91,118.02,0.956708,Nightlife,Restaurant,None,5.278115
2,Brent,Entire rental unit,1.5,2.0,30,117.59,0.481797,Grocery Store,None,None,4.430817
3,Kensington_and_Chelsea,Entire rental unit,1.0,2.0,91,118.02,0.398479,Restaurant,None,None,5.666427
4,Westminster,Entire rental unit,3.0,4.0,186,132.94,0.200253,Restaurant,Retail,None,6.804615


In [11]:
df_full_train, df_test = train_test_split(inside_airbnb_df, test_size=0.2, random_state=33, stratify=inside_airbnb_df['borough'])
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=33, stratify=df_full_train['borough'])

In [12]:
X_train = df_train.drop(['log_price'], axis=1)
y_train = df_train['log_price'].copy()
X_val = df_val.drop(['log_price'], axis=1)
y_val = df_val['log_price'].copy()
X_test = df_test.drop(['log_price'], axis=1)
y_test = df_test['log_price'].copy()

In [13]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

## Data pipeline

In [14]:
cat_attribs = ['borough', 'property_type', 'amenity_1', 'amenity_2', 'amenity_3']
num_attribs = ['bathrooms', 'bedrooms', 'minimum_nights', 'crime_rate', 'distance_to_station']

In [15]:
num_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
)

In [16]:
cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore', sparse_output=False),
)

In [17]:
preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs),
])

In [18]:
X_train_prepared = preprocessing.fit_transform(X_train)  # fitting only occurs here!
X_val_prepared = preprocessing.transform(X_val)          # using 'preprocessing' object to transform data frame
X_test_prepared = preprocessing.transform(X_test)        # using 'preprocessing' object to transform data frame

In [19]:
X_train_prepared_df = pd.DataFrame(
    data=X_train_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=X_train.index,
)

In [20]:
X_val_prepared_df = pd.DataFrame(
    data=X_val_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=X_val.index,
)

In [21]:
X_test_prepared_df = pd.DataFrame(
    data=X_test_prepared,
    columns=preprocessing.get_feature_names_out(),
    index=X_test.index,
)

In [22]:
print(f'Training size: {round(len(X_train_prepared_df)/len(inside_airbnb_df), 5):>10}')
print(f'Validation size: {round(len(X_val_prepared_df)/len(inside_airbnb_df), 5):>8}')
print(f'Testing size: {round(len(X_test_prepared_df)/len(inside_airbnb_df), 5):>11}')

Training size:    0.59946
Validation size:  0.20027
Testing size:     0.20027


## Predictive modeling
### Support vector regressor

In [23]:
svr = SVR()
svr.fit(X_train_prepared_df, y_train)

SVR()

In [24]:
y_pred_svr = svr.predict(X_val_prepared_df)
svr_rmse = root_mean_squared_error(y_val, y_pred_svr)
print(f'RMSE for support vector regressor: {round(svr_rmse, 5)}')

RMSE for support vector regressor: 0.49696


In [25]:
svr_r2_score = r2_score(y_val, y_pred_svr)
print(f'R2 for support vector regressor: {round(svr_r2_score, 5)}')

R2 for support vector regressor: 0.36559


## Cross validation
### Support vector regressor

In [26]:
svr_rmses = -cross_val_score(svr, X_train_prepared_df, y_train, scoring='neg_root_mean_squared_error', cv=10)

In [27]:
print(f"Mean and std dev: {pd.Series(svr_rmses).describe().loc['mean']:.5f} ± {pd.Series(svr_rmses).describe().loc['std']:.5f}")

Mean and std dev: 0.48424 ± 0.10824


## Grid search
### Support vector regressor

In [28]:
full_pipeline = Pipeline([
    # ('preprocessing', preprocessing),
    ('svr', SVR()),
])

In [29]:
param_grid = [
    {'svr__C': [0.001, 0.01, 0.1],
     'svr__epsilon': [0.001, 0.01, 0.1]},
    {'svr__C': [1, 10, 100],
     'svr__epsilon': [0.1, 1, 10]},
]

In [30]:
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3, error_score='raise',
                          scoring='neg_root_mean_squared_error')
grid_search.fit(X_train_prepared_df, y_train)

GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('svr', SVR())]),
             param_grid=[{'svr__C': [0.001, 0.01, 0.1],
                          'svr__epsilon': [0.001, 0.01, 0.1]},
                         {'svr__C': [1, 10, 100],
                          'svr__epsilon': [0.1, 1, 10]}],
             scoring='neg_root_mean_squared_error')

In [31]:
grid_search.best_params_

{'svr__C': 1, 'svr__epsilon': 0.1}

In [32]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res = cv_res[['param_svr__C', 'param_svr__epsilon', 'split0_test_score',
                 'split1_test_score', 'split2_test_score', 'mean_test_score']]

In [33]:
score_cols = ['split0', 'split1', 'split2', 'mean_test_rmse']
cv_res.columns = ['C', 'epsilon'] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round(5).astype(np.float64)

In [34]:
cv_res.head()

,C,epsilon,split0,split1,split2,mean_test_rmse
9,1.0,0.100,0.48238,0.48712,0.52000,0.49650
12,10.0,0.100,0.51345,0.52211,0.57629,0.53728
6,0.1,0.001,0.52996,0.53488,0.55517,0.54000
7,0.1,0.010,0.53007,0.53561,0.55462,0.54010
8,0.1,0.100,0.53108,0.53868,0.55622,0.54200
